In [11]:
import Config as C
import numpy as np
np.random.seed(C.SEED)
import random
random.seed(C.SEED)
from RewardFnSpace import *
import pickle
import more_itertools
from AcrobotUtils import *
from scipy.spatial.distance import pdist, squareform
import os
import os.path as osp
from A2C import *
from PlotUtils import *
from Eval import *
from rlpyt.samplers.serial.sampler import SerialSampler
from rlpyt.samplers.parallel.gpu.sampler import GpuSampler
from datetime import datetime

In [2]:

def findSamplesInTrajs (stateSamples, trajs) : 
    """ 
    For each state sample, find all indices (i, j) such that
    the jth state in ith trajectory is approximately the state
    sample
    """
    nSamples = stateSamples.shape[0]
    stateOccurenceIndices = [[] for _ in range(nSamples)]
    allStates = [np.stack([s for s, _, _ in t]) for t in trajs]
    for i, traj in enumerate(trajs) : 
        trajLen = len(traj)
        D = squareform(pdist(np.concatenate((stateSamples, allStates[i]), axis=0)))
        D = D[:nSamples, nSamples:]
        indices = np.where(D < C.STATE_SIMILARITY_THRESH)
        for j, k  in zip(*indices) : 
            stateOccurenceIndices[j].append((i, k))
    return stateOccurenceIndices


In [3]:
def generateStateSamples (trajs, nSamples) : 
    """ get the distribution of start states """
    allStates = [[s for s, _, _ in t] for t in trajs[:10]]
    allStates = list(more_itertools.flatten(allStates))
    states = random.sample(allStates, k=nSamples)
    states = np.array(states)
    return states

In [4]:
def estimateValueFromTrajs (stateIndices, trajs, rewardFn) :
    """ 
    Estimate the value for each state from expert 
    trajectories.
    """
    def computeReturnOnTraj (traj) : 
        R = [rewardFn(s) for s, _, _ in traj]
        return computeReturns(R, C.DISCOUNT)[0]
    values = []
    for i, indices in enumerate(stateIndices) : 
        truncatedTrajs = [trajs[i][j:] for i, j in indices] 
        vhat = np.mean([computeReturnOnTraj(t) for t in truncatedTrajs])
        values.append(vhat)
    return values

In [5]:
def estimateValueFromAgent (stateSamples, agent, rewardFn) : 
    """
    Use the learnt value function network through
    A2C to estimate value for states.
    """
    def estimateForState (s) : 
        cpus = list(range(C.N_PARALLEL))
        affinity = dict(cuda_idx=C.CUDA_IDX, workers_cpus=cpus)
        agent_ = CategoricalPgAgent(
            AcrobotNet, 
            initial_model_state_dict=agent.state_dict())
        sampler = SerialSampler(
            EnvCls=rlpyt_make,
            env_kwargs=dict(
                id=C.ENV, 
                reward=rewardFn, 
                internalStateFn=C.INTERNAL_STATE_FN, 
                s0=s),
            batch_T=C.HORIZON,
            batch_B=C.BATCH_B,
            max_decorrelation_steps=0,
        )
        sampler.initialize(
            agent=agent_,
            affinity=affinity,
            seed=C.SEED
        )
        _, traj_info = sampler.obtain_samples(0)
        returns = [t['DiscountedReturn'] for t in traj_info]
        return np.mean(returns)
    estimates = list(map(estimateForState, stateSamples))
    return estimates


In [6]:
def getAllTraj () : 
    """ get all trajectories from C.TRAJ_DIR """
    def loadPickle (f) : 
        with open(osp.join(C.TRAJ_DIR, f), 'rb') as fd : 
            return pickle.load(fd)
    return list(map(loadPickle, os.listdir(C.TRAJ_DIR)))

In [7]:
def irl (rewardFnSpace) :
    """
    Find the explanatory reward function for expert's 
    policy in the space of reward functions.
    """
    trajs = getAllTraj()
    stateSamples = generateStateSamples(trajs, C.IRL_STATE_SAMPLES)
    indices = findSamplesInTrajs(stateSamples, trajs) 
    for i in range(C.IRL_ITR) : 
        rewardFn = rewardFnSpace.current()
        agent = findOptimalAgent(rewardFn)
        env = rlpyt_make('Acrobot-v1', internalStateFn=C.INTERNAL_STATE_FN)
        expertValues = [estimateValueFromTrajs(indices, trajs, _) 
                        for _ in rewardFnSpace.rewardBases]
        inferiorValues = [estimateValueFromAgent(stateSamples, agent, _)
                          for _ in rewardFnSpace.rewardBases]
        rewardFnSpace.refine(expertValues, inferiorValues)
    return agent, rewardFn

In [8]:
# Chạy thuật toán IRL
agent, rewardFn = irl(RewardFnSpace(acrobotRewardBases(np.pi / 2, np.pi / 2)))

d:\ASUS\Trí tuệ nhân tạo nâng cao - CS211\Inverse Reinforcement Learning\IRL\a2c_acrobot-v1\run_0
D:\ASUS\Trí tuệ nhân tạo nâng cao - CS211\Inverse Reinforcement Learning\Inverse-Reinforcement-Learning\rlpyt\data
2024-11-14 10:29:22.303702  | a2c_acrobot-v1_0 Runner  master CPU affinity: [0, 1, 2, 3, 4, 5, 6, 7].
2024-11-14 10:29:22.304701  | a2c_acrobot-v1_0 Runner  master Torch threads: 4.
using seed 3260
2024-11-14 10:29:22.457821  | a2c_acrobot-v1_0 Sampler decorrelating envs, max steps: 400


c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym

2024-11-14 10:29:24.930146  | a2c_acrobot-v1_0 Serial Sampler initialized.
2024-11-14 10:29:24.931144  | a2c_acrobot-v1_0 Running 1872 iterations of minibatch RL.
2024-11-14 10:29:28.617870  | a2c_acrobot-v1_0 Optimizing over 312 iterations.


c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\rlpyt-0.1.2-py3.11.egg\rlpyt\algos\pg\a2c.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
0% [##############################] 100% | ETA: 00:00:00

2024-11-14 10:31:06.796075  | a2c_acrobot-v1_0 itr #311 saving snapshot...
2024-11-14 10:31:06.831336  | a2c_acrobot-v1_0 itr #311 saved
2024-11-14 10:31:06.831336  | -----------------------------  -------------
2024-11-14 10:31:06.843349  | Diagnostics/NewCompletedTrajs    199
2024-11-14 10:31:06.843349  | Diagnostics/StepsInTrajWindow  50000
2024-11-14 10:31:06.843349  | Diagnostics/Iteration            311
2024-11-14 10:31:06.843349  | Diagnostics/CumTime (s)           98.2115
2024-11-14 10:31:06.843349  | Diagnostics/CumSteps           99840
2024-11-14 10:31:06.843349  | Diagnostics/CumCompletedTrajs    199
2024-11-14 10:31:06.843349  | Diagnostics/CumUpdates           312
2024-11-14 10:31:06.843349  | Diagnostics/StepsPerSecond      1016.58
2024-11-14 10:31:06.843349  | Diagnostics/UpdatesPerSecond       3.17682
2024-11-14 10:31:06.843349  | Diagnostics/ReplayRatio            1
2024-11-14 10:31:06.858977  | Diagnostics/CumReplayRatio         1
2024-11-14 10:31:06.858977  | LengthA

...

2024-11-14 11:54:15.657411  | a2c_acrobot-v1_0 itr #1559 saving snapshot...
2024-11-14 11:54:15.664977  | a2c_acrobot-v1_0 itr #1559 saved
2024-11-14 11:54:15.669967  | -----------------------------  --------------
2024-11-14 11:54:15.670966  | Diagnostics/NewCompletedTrajs    1065
2024-11-14 11:54:15.672960  | Diagnostics/StepsInTrajWindow    9408
2024-11-14 11:54:15.674078  | Diagnostics/Iteration            1559
2024-11-14 11:54:15.675074  | Diagnostics/CumTime (s)           651.552
2024-11-14 11:54:15.676072  | Diagnostics/CumSteps           499200
2024-11-14 11:54:15.677070  | Diagnostics/CumCompletedTrajs    4907
2024-11-14 11:54:15.678067  | Diagnostics/CumUpdates           1560
2024-11-14 11:54:15.680063  | Diagnostics/StepsPerSecond        619.127
2024-11-14 11:54:15.682056  | Diagnostics/UpdatesPerSecond        1.93477
2024-11-14 11:54:15.683052  | Diagnostics/ReplayRatio             1
2024-11-14 11:54:15.684563  | Diagnostics/CumReplayRatio          1
2024-11-14 11:54:15.684


Total time elapsed: 00:02:41
0% [##############################] 100% | ETA: 00:00:00

2024-11-14 11:56:02.698197  | a2c_acrobot-v1_0 itr #1871 saving snapshot...
2024-11-14 11:56:02.705176  | a2c_acrobot-v1_0 itr #1871 saved
2024-11-14 11:56:02.710592  | -----------------------------  --------------
2024-11-14 11:56:02.711586  | Diagnostics/NewCompletedTrajs    1079
2024-11-14 11:56:02.712583  | Diagnostics/StepsInTrajWindow    9296
2024-11-14 11:56:02.713582  | Diagnostics/Iteration            1871
2024-11-14 11:56:02.714579  | Diagnostics/CumTime (s)           758.592
2024-11-14 11:56:02.715576  | Diagnostics/CumSteps           599040
2024-11-14 11:56:02.716573  | Diagnostics/CumCompletedTrajs    5986
2024-11-14 11:56:02.717571  | Diagnostics/CumUpdates           1872
2024-11-14 11:56:02.718567  | Diagnostics/StepsPerSecond        932.734
2024-11-14 11:56:02.719565  | Diagnostics/UpdatesPerSecond        2.91479
2024-11-14 11:56:02.720563  | Diagnostics/ReplayRatio             1
2024-11-14 11:56:02.721561  | Diagnostics/CumReplayRatio          1
2024-11-14 11:56:02.722


Total time elapsed: 00:01:46


2024-11-14 11:56:03.357057  | Sampler decorrelating envs, max steps: 0
2024-11-14 11:56:03.358856  | Serial Sampler initialized.
2024-11-14 11:56:06.323993  | Sampler decorrelating envs, max steps: 0
2024-11-14 11:56:06.323993  | Serial Sampler initialized.
2024-11-14 11:56:09.397379  | Sampler decorrelating envs, max steps: 0
2024-11-14 11:56:09.397379  | Serial Sampler initialized.
2024-11-14 11:56:12.409612  | Sampler decorrelating envs, max steps: 0
2024-11-14 11:56:12.411607  | Serial Sampler initialized.
2024-11-14 11:56:15.317600  | Sampler decorrelating envs, max steps: 0
2024-11-14 11:56:15.318598  | Serial Sampler initialized.
2024-11-14 11:56:18.323441  | Sampler decorrelating envs, max steps: 0
2024-11-14 11:56:18.323441  | Serial Sampler initialized.
2024-11-14 11:56:21.260013  | Sampler decorrelating envs, max steps: 0
2024-11-14 11:56:21.261011  | Serial Sampler initialized.
2024-11-14 11:56:24.123325  | Sampler decorrelating envs, max steps: 0
2024-11-14 11:56:24.139893

In [12]:
# Tạo thư mục results trong cùng thư mục với file code
current_dir = os.getcwd()
save_dir = osp.join(current_dir, 'results')
os.makedirs(save_dir, exist_ok=True)

# Tạo dữ liệu để vẽ
xRange = np.arange(-np.pi, np.pi, 0.1)
yRange = np.arange(-np.pi, np.pi, 0.1)
toExternal = lambda x, y : toExternalStateRep([x, y, 0, 0])
RFn = compose(rewardFn, toExternal)

# Vẽ đồ thị
plotFunction(RFn, xRange, yRange, 'theta1', 'theta2', 'R')

# Tạo tên file với timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'recovered_{timestamp}.png'
save_path = osp.join(save_dir, filename)

# Lưu hình với chất lượng cao
plt.savefig(save_path, 
            dpi=300,           # độ phân giải cao
            bbox_inches='tight',  # cắt bỏ viền trắng thừa
            pad_inches=0.1,    # thêm padding nhỏ
            format='png')      # định dạng file

# Hiển thị đồ thị
plt.show()
from Eval import simulateAgent_2
# Chạy mô phỏng
simulateAgent_2(agent, render=True)
# video_display, steps = simulateAgentFile(agent)

TypeError: FigureBase.gca() got an unexpected keyword argument 'projection'

<Figure size 640x480 with 0 Axes>